In [1]:
from src.field import Field
from src.base import sql,BaseSQL
from typing import _GenericAlias

DATATYPES = {
    'int':'INTEGER',
    'str':'TEXT',
    'float':'REAL',
    'bytes':'BLOB'
}


class BaseModel(BaseSQL):

    def __init__(self) -> None:

        self.__name__ = self.__class__.__name__.lower()
        self.__columns__ = list(self.__class__.__annotations__.keys())
        self.__column_info__ = {key:getattr(self,key).__info__() for key in self.columns()}

        self._field_info = []
        for attr in self.__columns__:
            assert type(getattr(self,attr)) == Field, f'{attr} Must be a field type in order to use BaseModel'
            self._field_info.append(getattr(self,attr))
        self._field_info = tuple(self._field_info)
        #check if matches 
        for key,val in self.__class__.__annotations__.items():
            assert isinstance(getattr(self,key).__value__,val.__args__[0]),f'Invalid data type {val.__args__[0]} must match with default {type(getattr(self,key).__value__)} type'

    def __sql__(self) -> str:

        columns = []

        for key,val in self.__class__.__annotations__.items():
            getattr(self,key).type = val.__args__[0]
            getattr(self,key).name = key
            if isinstance(val,_GenericAlias):
                columns.append(f'{key} {DATATYPES[val.__args__[0].__name__]} {getattr(self.__class__(),key).__sql__()}')

        return f"CREATE TABLE {self.__class__.__name__.lower()} ( {', '.join(columns)} );"
class TableTest(BaseModel):
    name:Field[int] = Field(default=3,primary_key=True)
    address:Field[str] = Field(default='5',unique=True,required=True)

In [2]:
val = TableTest() # 1
sql(val)

'CREATE TABLE tabletest ( name INTEGER PRIMARY KEY, address TEXT NOT NULL UNIQUE );'